In [1]:
import pandas as pd
from pathlib import Path  
import sys
import sqlite3

CUT_OF_UNIQUE_VALUES=40

In [39]:
data_filtered_cedd=pd.read_csv("data_filtered_with_cedd.csv")
df_counts  = pd.DataFrame(columns = ['col', 'count'])
# for col in data_filtered_cedd:
#     no_of_unique_values=data_filtered_cedd[col].nunique()
#     df_counts.loc[len(df_counts)]=[col,no_of_unique_values]
# #     if( not col.startswith( 'Unnamed' ) && no_of_unique_values<20):
# #         print("%s--->%s"%(col,))
# df_counts = df_counts.sort_values(by=['count'], ascending=False)
# display(df_counts.head(30))



for col in data_filtered_cedd:
    no_of_unique_values=data_filtered_cedd[col].nunique()
    if( not col.startswith( 'Unnamed' ) and no_of_unique_values<40):

        data_filtered_cedd[col]=pd.Categorical(data_filtered_cedd[col])
                
data_filtered_cedd.dtypes.value_counts()
    



category    40
object      12
int64        6
float64      4
category     4
            ..
category     1
category     1
category     1
category     1
category     1
Length: 87, dtype: int64